In [1]:
from PIL import Image, ImageDraw
import random
import math

In [2]:
def generate_arrow_image(
    width: int = 320,
    height: int = 320,
    min_length: float = 50,
    max_length: float = 150,
    base_angle_deg: float = 180,
    angle_variation_deg: float = 20,
    shaft_width: int = 3,
    head_length: float = 10,
    head_half_width: float = 5
) -> Image.Image:
    """
    生成一张白底黑箭头的图像，箭头主方向在 base_angle_deg 附近随机偏转 ±angle_variation_deg。

    参数：
        width, height:  画布大小（像素）。
        min_length, max_length:  箭头杆（shaft）长度的随机范围（像素）。
        base_angle_deg:  箭头主方向（度），比如 180 表示正左。
        angle_variation_deg:  在主方向基础上随机偏转的幅度（度）。
        shaft_width:  箭头杆的线宽（像素）。
        head_length:  箭头头部的“长度”，从箭尖往回量（像素）。
        head_half_width:  箭头头部底边的半宽度（像素）。

    返回：
        一张 PIL.Image.Image 对象，背景白色，包含一支黑色箭头。
    """
    while True:
        # 1. 随机确定箭头长度 L 和角度 θ（弧度）
        L = random.uniform(min_length, max_length)
        theta_deg = random.uniform(base_angle_deg - angle_variation_deg,
                                   base_angle_deg + angle_variation_deg)
        theta = math.radians(theta_deg)

        # 2. 根据 L 和 θ 计算方向向量 (dx, dy) 以及单位向量 (ux, uy)
        dx = L * math.cos(theta)
        dy = L * math.sin(theta)
        dist = math.hypot(dx, dy)
        ux = dx / dist
        uy = dy / dist

        # 3. 计算箭头可放置的“箭尖”（head）坐标范围
        #    要求：箭尖 (x2, y2) 必须满足以下两条：
        #      a) 0 ≤ x2 ≤ width,  0 ≤ y2 ≤ height
        #      b) 箭尾 (x1 = x2 - dx, y1 = y2 - dy) 也要在 [0, width]×[0, height]
        #    因此：
        #      0 ≤ x2 ≤ width
        #      0 ≤ x2 - dx ≤ width  →  dx ≤ x2 ≤ width + dx
        #    联合可得：
        #      x2_min = max(0, dx),  x2_max = min(width, width + dx)
        #
        #    同理 y2_min = max(0, dy),  y2_max = min(height, height + dy)
        x2_min = max(0, dx)
        x2_max = min(width, width + dx)
        y2_min = max(0, dy)
        y2_max = min(height, height + dy)

        # 如果范围无效，就重试
        if x2_min > x2_max or y2_min > y2_max:
            continue

        # 4. 随机在可行区域内选取箭尖 (x2, y2)
        x2 = random.uniform(x2_min, x2_max)
        y2 = random.uniform(y2_min, y2_max)

        # 5. 计算箭尾 (x1, y1)
        x1 = x2 - dx
        y1 = y2 - dy

        # 6. 计算箭头头部三角形的三个顶点
        #    (1) 箭尖顶点就是 (x2, y2)
        #    (2) 箭头底边中心(base)：base = (x2 - head_length * ux,  y2 - head_length * uy)
        base_x = x2 - head_length * ux
        base_y = y2 - head_length * uy

        #    (3) 计算垂直主方向的法向量 (px, py) = (-uy, ux)
        px = -uy
        py = ux

        #    箭头底边左右两个顶点
        left_x  = base_x + head_half_width * px
        left_y  = base_y + head_half_width * py
        right_x = base_x - head_half_width * px
        right_y = base_y - head_half_width * py

        # 7. 检查：箭尾、箭尖、以及箭头底边两个顶点是否都在画布 [0, width]×[0, height] 内
        all_x = [x1, x2, left_x, right_x]
        all_y = [y1, y2, left_y, right_y]
        if (min(all_x) < 0 or max(all_x) > width or
            min(all_y) < 0 or max(all_y) > height):
            # 如果任何一点跑出了边界，就重新随机
            continue

        # 如果到这里都没问题，就跳出循环，准备绘图
        break

    # 8. 创建白底画布并绘制
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)

    # 绘制箭头杆（shaft）
    draw.line((x1, y1, x2, y2), fill='black', width=shaft_width)

    # 绘制箭头头部三角形
    draw.polygon([
        (x2, y2),
        (left_x, left_y),
        (right_x, right_y)
    ], fill='black')

    return img

In [9]:
# 生成 20 张图片，保存为 arrow_0.png, arrow_1.png, ..., arrow_19.png
for i in range(5):
    img = generate_arrow_image(base_angle_deg=180, shaft_width= 5, head_length=20, head_half_width= 15)
    img.save(f'arrow_{i}.png')